<a href="https://colab.research.google.com/github/razymawardi/portfolio/blob/main/Mini_Project_Data_Manipulation_with_Pandas_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mini Project - Data Manipulation with Pandas Part 1**

# **Pendahuluan**

Pada proyek kali ini kita akan melakukan proses ETL yang biasanya dilakukan sebelum analisis data. Proses ini dilakukan karena biasanya data yang dibutuhkan masih belum bersih, sehingga tidak bisa dilakukan analisis lebih lanjut. Untuk itu proses ini menjadi sangat penting karena jika tidak dilakukan dengan baik maka akan berpengaruh buruk pada hasil analisis yang kita lakukan.

Data yang digunakan adalah data perusahaan ritel fiktif DQLab yang diantaranya berisi kumpulan data untuk setiap kota dan provinsi, tanggal order, customer, jenis order terkait brand, product, quantity dan item price serta GMV/Gross Merchandise Volume (total price).

Diberikan dataset *‘retail_raw_test.csv’*, berikut adalah langkah pengerjaan yang akan kita lakukan dalam proyek kali ini:

1. Baca dataset
2. Tipe data diubah menjadi tipe yang seharusnya
  * customer_id dari string ke int64,
  * quantity dari string ke int64,
  * item_price dari string ke int64
3. transform product_value supaya bentuknya seragam dengan format PXXXX, assign ke kolom baru "product_id", dan drop kolom "product_value", jika terdapat nan gantilah dengan "unknown".
4. tranform order_date menjadi value dengan format YYYY-mm-dd
5. cek data hilang dari tiap kolom dan kemudian isi missing value
  * di brand dengan "no_brand", dan
  * cek dulu bagaimana missing value di city & province - isi missing value di city dan province dengan "unknown"
6. create column city/province dari gabungan city & province
7. membuat index berdasarkan city_provice, order_date, customer_id, order_id, 8 product_id (cek index)
8. membuat kolom "total_price" sebagai hasil perkalian quantity dengan item_price
9. slice data hanya untuk Jan 2019

**Notes** :

Dataset : https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/retail_raw_test.csv


# **1. Baca dan simpan dataset**

In [1]:
import pandas as pd

In [2]:
# # 0. Mengambil dan menyimpan dataset (Optional, jika file "retail_raw_test.csv" sudah didownload maka langkah ini bisa dilewati)
# print("[0] SIMPAN DATASET")
# data = pd.read_csv("https://dqlab-dataset.s3-ap-southeast-1.amazonaws.com/retail_raw_test.csv", low_memory=False)
# print(data.head(3))
# data.to_csv('retail_raw_test.csv')

In [3]:
import pandas as pd
# 1. Baca dataset
print("[1] BACA DATASET")
df = pd.read_csv("retail_raw_test.csv")
print("    Dataset:\n", df.head())
print("    Info:\n", df.info())

[1] BACA DATASET
    Dataset:
    Unnamed: 0  order_id    order_date customer_id           city     province  \
0           0   1730350  Dec 11, 2019      '13447      Surakarta  Jawa Tengah   
1           1   1677490  Jul 31, 2019          '0            NaN          NaN   
2           2   1704211  Oct 18, 2019      '16128  Jakarta Pusat  DKI Jakarta   
3           3   1679695  Aug 07, 2019      '16225     Yogyakarta   Yogyakarta   
4           4   1679080  Aug 05, 2019          '0            NaN          NaN   

     brand quantity item_price  product_value  
0  BRAND_F      '24    '113000         1374.0  
1  BRAND_F       '1   '1164000         1370.0  
2  BRAND_H      '12    '747000         1679.0  
3  BRAND_H       '6    '590000         1708.0  
4  BRAND_E       '2    '740000         1201.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  

# **2. Ubah tipe data**

In [4]:
# 2. Ubah tipe data
print("\n[2] UBAH TIPE DATA")
df["customer_id"] = df["customer_id"].apply(lambda x: x.split("'")[1]).astype("int64")
df["quantity"] = df["quantity"].apply(lambda x: x.split("'")[1]).astype("int64")
df["item_price"] = df["item_price"].apply(lambda x: x.split("'")[1]).astype("int64")
print("    Tipe data:\n", df.dtypes)


[2] UBAH TIPE DATA
    Tipe data:
 Unnamed: 0         int64
order_id           int64
order_date        object
customer_id        int64
city              object
province          object
brand             object
quantity           int64
item_price         int64
product_value    float64
dtype: object


In [5]:
print(df.city.unique())
print(df.province.unique())

['Surakarta' nan 'Jakarta Pusat' 'Yogyakarta' 'Denpasar' 'Tangerang'
 'Bandung' 'Jakarta Selatan' 'Depok' 'Jakarta Utara' 'Palangkaraya'
 'unknown' 'Bogor' 'Purwokerto' 'Pekanbaru' 'Bekasi' 'Bandar Lampung'
 'Makassar' 'Jakarta Timur' 'Semarang' 'Jakarta Barat' 'Lubuklinggau'
 'Tebing Tinggi' 'Surabaya' 'Malang' 'Bukittinggi' 'Manado' 'Ubud' 'Palu'
 'Medan' 'Palembang' 'Banjarmasin' 'Banda Aceh' 'Jambi' 'Pangkalpinang'
 'Padang' 'Binjai' 'Pematang Siantar' 'Pontianak']
['Jawa Tengah' nan 'DKI Jakarta' 'Yogyakarta' 'Bali' 'Banten' 'Jawa Barat'
 'Kalimantan Tengah' 'unknown' 'Riau' 'Lampung' 'Sulawesi Selatan'
 'Sumatra Selatan' 'Sumatra Utara' 'Jawa Timur' 'Sumatra Barat'
 'Sulawesi Utara' 'Sulawesi Tengah' 'Kalimantan Selatan' 'Aceh' 'Jambi'
 'Bangka Belitung' 'Kalimantan Barat']


# **3. Transform "product_value" ke format "PXXXX"**

In [6]:
# 3. Transform "product_value" supaya bentuknya seragam dengan format "PXXXX", 
# assign ke kolom baru "product_id", dan drop kolom "product_value", 
# jika terdapat nan gantilah dengan "unknown"

print("\n[3] TRANSFORM product_value MENJADI product_id")
# Buat fungsi
import math
def impute_product_value(val):
    if math.isnan(val):
        return "unknown"
    else:
        return "P" + f"{str(val).split('.')[0]:0>4}"
# Buat kolom "product_id"
df["product_id"] = df["product_value"].apply(lambda x: impute_product_value(x))
# Hapus kolom "product_value"
df.drop(["product_value"], axis=1, inplace=True)
# Cetak 5 data teratas
print(df.head())


[3] TRANSFORM product_value MENJADI product_id
   Unnamed: 0  order_id    order_date  customer_id           city  \
0           0   1730350  Dec 11, 2019        13447      Surakarta   
1           1   1677490  Jul 31, 2019            0            NaN   
2           2   1704211  Oct 18, 2019        16128  Jakarta Pusat   
3           3   1679695  Aug 07, 2019        16225     Yogyakarta   
4           4   1679080  Aug 05, 2019            0            NaN   

      province    brand  quantity  item_price product_id  
0  Jawa Tengah  BRAND_F        24      113000      P1374  
1          NaN  BRAND_F         1     1164000      P1370  
2  DKI Jakarta  BRAND_H        12      747000      P1679  
3   Yogyakarta  BRAND_H         6      590000      P1708  
4          NaN  BRAND_E         2      740000      P1201  


# **4. Tranform order_date menjadi value dengan format "YYYY-mm-dd"**

In [7]:
# 4. Tranform order_date menjadi value dengan format "YYYY-mm-dd"
print("\n[4] TRANSFORM order_date MENJADI FORMAT YYYY-mm-dd")
months_dict = {
   "Jan":"01",
   "Feb":"02",
   "Mar":"03",
   "Apr":"04",
   "May":"05",
   "Jun":"06",
   "Jul":"07",
   "Aug":"08",
   "Sep":"09",
   "Oct":"10",
   "Nov":"11",
   "Dec":"12"
}
df["order_date"] = pd.to_datetime(df["order_date"].apply(lambda x: str(x)[-4:] + "-" + months_dict[str(x)[:3]] + "-" + str(x)[4:7]))
print("    Tipe data:\n", df.dtypes)


[4] TRANSFORM order_date MENJADI FORMAT YYYY-mm-dd
    Tipe data:
 Unnamed: 0              int64
order_id                int64
order_date     datetime64[ns]
customer_id             int64
city                   object
province               object
brand                  object
quantity                int64
item_price              int64
product_id             object
dtype: object


# **5. Mengatasi data yang hilang di beberapa kolom**

In [8]:
# 5. Mengatasi data yang hilang di beberapa kolom
print("\n[5] HANDLING MISSING VALUE")
# Kolom "city" dan "province" masih memiliki missing value, nilai yang hilang di kedua kolom ini diisi saja dengan "unknown"
df[["city","province"]] = df[["city","province"]].fillna("unknown")
# Kolom brand juga masih memiliki missing value, Ganti value NaN menjadi "no_brand"
df["brand"] = df["brand"].fillna("no_brand")
# Cek apakah masih terdapat missing value di seluruh kolom 
print("    Info:\n", df.info())



[5] HANDLING MISSING VALUE
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Unnamed: 0   5000 non-null   int64         
 1   order_id     5000 non-null   int64         
 2   order_date   5000 non-null   datetime64[ns]
 3   customer_id  5000 non-null   int64         
 4   city         5000 non-null   object        
 5   province     5000 non-null   object        
 6   brand        5000 non-null   object        
 7   quantity     5000 non-null   int64         
 8   item_price   5000 non-null   int64         
 9   product_id   5000 non-null   object        
dtypes: datetime64[ns](1), int64(5), object(4)
memory usage: 390.8+ KB
    Info:
 None


# **6. Membuat kolom baru "city/province"**

In [9]:
# 6. Membuat kolom baru "city/province" dengan menggabungkan kolom "city" dan kolom "province" dan delete kolom asalnya
print("\n[6] MEMBUAT KOLOM BARU city/province")
df["city/province"] = df["city"] + "/" + df["province"]
# drop kolom "city" dan "province" karena telah digabungkan
df.drop(["city","province"], axis=1, inplace=True)
# Cetak 5 data teratas
print(df.head())


[6] MEMBUAT KOLOM BARU city/province
   Unnamed: 0  order_id order_date  customer_id    brand  quantity  \
0           0   1730350 2019-12-11        13447  BRAND_F        24   
1           1   1677490 2019-07-31            0  BRAND_F         1   
2           2   1704211 2019-10-18        16128  BRAND_H        12   
3           3   1679695 2019-08-07        16225  BRAND_H         6   
4           4   1679080 2019-08-05            0  BRAND_E         2   

   item_price product_id              city/province  
0      113000      P1374      Surakarta/Jawa Tengah  
1     1164000      P1370            unknown/unknown  
2      747000      P1679  Jakarta Pusat/DKI Jakarta  
3      590000      P1708      Yogyakarta/Yogyakarta  
4      740000      P1201            unknown/unknown  


# **7. Membuat hierarchical index**

In [10]:
# 7. Membuat hierarchical index yang terdiri dari kolom "city/province", "order_date", "customer_id", "order_id", "product_id"
print("\n[7] MEMBUAT HIERACHICAL INDEX")
df = df.set_index(["city/province","order_date","customer_id","order_id","product_id"])
# urutkanlah berdasarkan index yang baru
df = df.sort_index()
# Cetak 5 data teratas
print(df.head())


[7] MEMBUAT HIERACHICAL INDEX
                                                                   Unnamed: 0  \
city/province          order_date customer_id order_id product_id               
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936             3957   
                       2019-11-12 12360       1715116  P0758             1445   
                                                       P3042              840   
                       2019-12-09 12374       1729036  P1660             1810   
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628             4514   

                                                                     brand  \
city/province          order_date customer_id order_id product_id            
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936       BRAND_K   
                       2019-11-12 12360       1715116  P0758       BRAND_C   
                                                       P3042       BRAND_R   
           

In [11]:
df.columns

Index(['Unnamed: 0', 'brand', 'quantity', 'item_price'], dtype='object')

# **8. Membuat kolom "total_price"**

In [12]:
# 8. Membuat kolom "total_price" yang formula nya perkalian antara kolom "quantity" dan kolom "item_price"
print("\n[8] MEMBUAT KOLOM total_price")
df["total_price"] = df["quantity"] * df["item_price"]
# Cetak 5 data teratas
print(df.head())


[8] MEMBUAT KOLOM total_price
                                                                   Unnamed: 0  \
city/province          order_date customer_id order_id product_id               
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936             3957   
                       2019-11-12 12360       1715116  P0758             1445   
                                                       P3042              840   
                       2019-12-09 12374       1729036  P1660             1810   
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628             4514   

                                                                     brand  \
city/province          order_date customer_id order_id product_id            
Banda Aceh/Aceh        2019-04-17 12818       1642480  P1936       BRAND_K   
                       2019-11-12 12360       1715116  P0758       BRAND_C   
                                                       P3042       BRAND_R   
           

# **9. Slice dataset agar hanya terdapat data bulan Januari 2019**

In [13]:
# 9. Slice dataset agar hanya terdapat data bulan Januari 2019
print("\n[9] SLICE DATASET UNTUK BULAN JANUARI 2019 SAJA")
idx = pd.IndexSlice
df_jan2019 = df.loc[idx[:,"2019-01-01":"2019-01-31"], :]
print("Dataset akhir:\n")
df_jan2019




[9] SLICE DATASET UNTUK BULAN JANUARI 2019 SAJA
Dataset akhir:



Unnamed: 0  \
city/province          order_date customer_id order_id product_id               
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628             4514   
Bandung/Jawa Barat     2019-01-09 16134       1617055  P1597             1744   
                       2019-01-10 17392       1617952  P2137             1418   
                       2019-01-14 15527       1618828  P3115             3890   
                       2019-01-29 13253       1620289  P0099             1788   
...                                                                       ...   
unknown/unknown        2019-01-30 0           1620766  P3070              312   
                                                       P3483             3063   
                       2019-01-31 0           1621057  P1298              549   
                                                       P1773             4105   
                                                       P2877             4083   

                                                                     brand  \
city/province          order_date customer_id order_id product_id            
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628       BRAND_C   
Bandung/Jawa Barat     2019-01-09 16134       1617055  P1597       BRAND_G   
                       2019-01-10 17392       1617952  P2137       BRAND_M   
                       2019-01-14 15527       1618828  P3115       BRAND_S   
                       2019-01-29 13253       1620289  P0099       BRAND_A   
...                                                                    ...   
unknown/unknown        2019-01-30 0           1620766  P3070       BRAND_R   
                                                       P3483       BRAND_S   
                       2019-01-31 0           1621057  P1298       BRAND_F   
                                                       P1773       BRAND_H   
                                                       P2877       BRAND_R   

                                                                   quantity  \
city/province          order_date customer_id order_id product_id             
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628             12   
Bandung/Jawa Barat     2019-01-09 16134       1617055  P1597              9   
                       2019-01-10 17392       1617952  P2137              2   
                       2019-01-14 15527       1618828  P3115              1   
                       2019-01-29 13253       1620289  P0099             12   
...                                                                     ...   
unknown/unknown        2019-01-30 0           1620766  P3070              1   
                                                       P3483              3   
                       2019-01-31 0           1621057  P1298              1   
                                                       P1773              5   
                                                       P2877              1   

                                                                   item_price  \
city/province          order_date customer_id order_id product_id               
Bandar Lampung/Lampung 2019-01-15 12515       1619257  P0628           695000   
Bandung/Jawa Barat     2019-01-09 16134       1617055  P1597           520000   
                       2019-01-10 17392       1617952  P2137          1062000   
                       2019-01-14 15527       1618828  P3115          1045000   
                       2019-01-29 13253       1620289  P0099           450000   
...                                                                       ...   
unknown/unknown        2019-01-30 0           1620766  P3070           593000   
                                                       P3483           593000   
                       2019-01-31 0           1621057  P1298           296000   
                                                       P1773           593000  

# **Notes**

Dapat diperhatikan pada langkah ketiga, kita mencari tahu terlebih dahulu nilai null dan tidak langsung di convert ke string. Hal ini dilakukan setelah melihat hasil dari `df.info()` yang menunjukkan bahwa masih ada yang kosong di kolom **'product_value'** yang jika di convert to string, nantinya value `NaN` akan berubah menjadi string `‘nan’`, kemudian ketika di tambahkan 0 di depan dan `concat` dengan char `'P'`, hasilnya akan menjadi `'P0nan'` yang tentu saja akan terlihat sangat aneh.

 

Berikutnya, pada langkah ke-4. Mengapa tidak langsung menggunakan kolom date yang sudah ada. Bukankah format waktunya sudah ideal?

Jawabannya: Tidak semua format datetime yang ideal pada umumnya akan ideal di dalam pandas environment. jadi harus di-translate dulu menjadi format yang ideal di dalam pandas sehingga pandas bisa mengenali data tersebut.
